In [1]:
import pandas as pd

In [2]:
pd.read_excel('proteínas glândula aranha.xltx').head()

,"Arachinidar e pep go, Samples Report created on 04/10/2016",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61
0,Arachinidar e pep go,Displaying:Total Spectrum Count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shotgun Proteomics
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#,Visible?,Starred?,Identified Proteins (632),Accession Number,Molecular Weight,Protein Grouping Ambiguity,Quantitative Variance,Taxonomy,biological adhesion,...,molecular function,molecular transducer activity,motor activity,nutrient reservoir activity,protein tag,structural molecule activity,transcription regulator activity,translation regulator activity,transporter activity,Mudpit_DATA.TXT (F003448 TUB NCBInr)
3,1,true,Empty,Cluster of PREDICTED: CAD protein [Tribolium c...,gi|91090153 [4],244 kDa,true,NaN,Tribolium castaneum,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
4,1.1,true,Empty,PREDICTED: CAD protein [Tribolium castaneum],gi|91090153,244 kDa,true,NaN,Tribolium castaneum,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [3]:
pd.read_excel('proteínas glândula aranha.xltx', skiprows=[0,1,2]).head()

,#,Visible?,Starred?,Identified Proteins (632),Accession Number,Molecular Weight,Protein Grouping Ambiguity,Quantitative Variance,Taxonomy,biological adhesion,...,molecular function,molecular transducer activity,motor activity,nutrient reservoir activity,protein tag,structural molecule activity,transcription regulator activity,translation regulator activity,transporter activity,Mudpit_DATA.TXT (F003448 TUB NCBInr)
0,1,True,Empty,Cluster of PREDICTED: CAD protein [Tribolium c...,gi|91090153 [4],244 kDa,True,NaN,Tribolium castaneum,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0
1,1.1,True,Empty,PREDICTED: CAD protein [Tribolium castaneum],gi|91090153,244 kDa,True,NaN,Tribolium castaneum,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
2,1.2,True,Empty,"CAD(r), partial [Bembidion planatum]",gi|382936963 (+1),32 kDa,True,NaN,Bembidion planatum,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,1.3,True,Empty,"CAD(r), partial [Bembidion proprium]",gi|382936975,33 kDa,True,NaN,Bembidion proprium,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,1.4,True,Empty,"CAD(r), partial [Bembidion xanthacrum]",gi|382937095,32 kDa,True,NaN,Bembidion xanthacrum,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [4]:
annotations = pd.read_excel('proteínas glândula aranha.xltx', skiprows=[0,1,2])

In [5]:
annotations.columns

Index(['#', 'Visible?', 'Starred?', 'Identified Proteins (632)',
       'Accession Number', 'Molecular Weight', 'Protein Grouping Ambiguity',
       'Quantitative Variance', 'Taxonomy', 'biological adhesion',
       'biological regulation', 'cell killing', 'cellular process',
       'developmental process', 'establishment of localization', 'growth',
       'immune system process', 'localization', 'locomotion',
       'metabolic process', 'multi-organism process',
       'multicellular organismal process', 'pigmentation', 'reproduction',
       'reproductive process', 'response to stimulus', 'rhythmic process',
       'viral reproduction', 'Golgi apparatus', 'cytoplasm', 'cytoskeleton',
       'endoplasmic reticulum', 'endosome', 'extracellular region',
       'intracellular organelle', 'membrane', 'mitochondrion', 'nucleus',
       'organelle membrane', 'organelle part', 'plasma membrane', 'ribosome',
       'antioxidant activity', 'auxiliary transport protein activity',
       'bindin

cols = {}

- **Biological Process**: biological adhesion, biological regulation, cell killing, cellular process, developmental process, establishment of localization, growth, immune system process, localization, locomotion, metabolic process, multi-organism process, multicellular organismal process, response to stimulus, rhythmic process, antioxidant activity, auxiliary transport protein activity, binding, catalytic activity, chemorepellent activity, electron carrier activity, enzyme regulator activity, molecular transducer activity, motor activity, nutrient reservoir activity, structural molecule activity, transcription regulator activity, translation regulator activity, transporter activity

- **Cellular component**: Golgi apparatus, cytoplasm, cytoskeleton, endoplasmic reticulum, endosome, extracellular region, intracellular organelle, membrane, mitochondrion, nucleus, organelle membrane, organelle part, plasma membrane, ribosome 

- **Molecular Function**: molecular function

In [38]:
cols['biological process'] = """biological adhesion, biological regulation, cell killing, cellular process, developmental process, establishment of localization, growth, immune system process, localization, locomotion, metabolic process, multi-organism process, multicellular organismal process, response to stimulus, rhythmic process, antioxidant activity, auxiliary transport protein activity, binding, catalytic activity, chemorepellent activity, electron carrier activity, enzyme regulator activity, molecular transducer activity, motor activity, nutrient reservoir activity, structural molecule activity, transcription regulator activity, translation regulator activity, transporter activity""".split(', ')
cols['cellular component'] = """Golgi apparatus, cytoplasm, cytoskeleton, endoplasmic reticulum, endosome, extracellular region, intracellular organelle, membrane, mitochondrion, nucleus, organelle membrane, organelle part, plasma membrane, ribosome""".split(', ')
cols['molecular function'] = ['molecular function'] 

In [39]:
set([x for l in cols.values() for x in l ]) - set(annotations.columns)

set()

In [40]:
set(annotations.columns) - set([x for l in cols.values() for x in l ]) 

{'#',
 'Accession Number',
 'Identified Proteins (632)',
 'Molecular Weight',
 'Mudpit_DATA.TXT (F003448 TUB NCBInr)',
 'Protein Grouping Ambiguity',
 'Quantitative Variance',
 'Starred?',
 'Taxonomy',
 'Visible?',
 'chaperone regulator activity',
 'chemoattractant activity',
 'metallochaperone activity',
 'pigmentation',
 'protein tag',
 'reproduction',
 'reproductive process',
 'viral reproduction'}